<a href="https://www.kaggle.com/code/kaiyoo88/tutorial-dacon-starter-code?scriptVersionId=236572192" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dacon-company-success-prediction/sample_submission.csv
/kaggle/input/dacon-company-success-prediction/train.csv
/kaggle/input/dacon-company-success-prediction/test.csv


In [10]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

In [7]:
data_path = "/kaggle/input/dacon-company-success-prediction"
train = pd.read_csv(f"{data_path}/train.csv")
test = pd.read_csv(f"{data_path}/test.csv")
sample_submission = pd.read_csv(f"{data_path}/sample_submission.csv")

In [5]:
train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8
2,TRAIN_0002,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,0.5
3,TRAIN_0003,2016,CT006,NaN,Seed,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,0.7
4,TRAIN_0004,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,0.1


In [8]:
# -------------------- 1. ID 컬럼 제거 --------------------
train = train.drop(columns=['ID'], axis=1)
test = test.drop(columns=['ID'], axis=1)

In [11]:
# -------------------- 2. 설립연도는 범주형으로 --------------------
train['설립연도'] = train['설립연도'].astype('object')
test['설립연도'] = test['설립연도'].astype('object')

# -------------------- 3. 컬럼 리스트 정의 --------------------
category_features = ['설립연도', '국가', '분야', '투자단계', '기업가치(백억원)']
numeric_features = ['직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)']
bool_features = ['인수여부', '상장여부']

# -------------------- 4. 결측치 컬럼 탐색 --------------------
print("결측치 개수 (train set):")
print(train.isnull().sum())

결측치 개수 (train set):
설립연도                 0
국가                   0
분야                 857
투자단계                 0
직원 수               174
인수여부                 0
상장여부                 0
고객수(백만명)          1320
총 투자금(억원)            0
연매출(억원)              0
SNS 팔로워 수(백만명)       0
기업가치(백억원)         1220
성공확률                 0
dtype: int64


In [12]:
# -------------------- 5. 범주형 데이터 인코딩 --------------------
encoders = {}

for feature in category_features:
    encoders[feature] = LabelEncoder()
    train[feature] = train[feature].fillna('Missing')
    test[feature] = test[feature].fillna('Missing')
    train[feature] = encoders[feature].fit_transform(train[feature])
    test[feature] = encoders[feature].transform(test[feature])

# -------------------- 6. 불리언 데이터 인코딩 --------------------
bool_map = {'Yes': 1, 'No': 0}
for feature in bool_features:
    train[feature] = train[feature].map(bool_map)
    test[feature] = test[feature].map(bool_map)


/tmp/ipykernel_31/2866482017.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train[feature] = train[feature].fillna('Missing')
/tmp/ipykernel_31/2866482017.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[feature] = test[feature].fillna('Missing')


In [13]:
# -------------------- 7. 결측치 채우기 (KNN Imputer 사용) --------------------
# 모든 feature를 대상으로 채운다
imputer = KNNImputer(n_neighbors=5)

# '성공확률' 컬럼은 제외하고 impute
X_train_full = train.drop(columns=['성공확률'])
X_train_full_imputed = pd.DataFrame(imputer.fit_transform(X_train_full), columns=X_train_full.columns)

X_test_imputed = pd.DataFrame(imputer.transform(test), columns=test.columns)

# 타겟은 따로
y = train['성공확률']

In [14]:
# -------------------- 8. 모델 학습 및 예측 --------------------
features = X_train_full_imputed.columns.tolist()

# KFold 교차검증
N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

models = []
cv_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train_full_imputed)):
    print(f"Fold {fold+1}")
    
    X_tr = X_train_full_imputed.iloc[train_idx]
    y_tr = y.iloc[train_idx]
    X_val = X_train_full_imputed.iloc[valid_idx]
    y_val = y.iloc[valid_idx]
    
    model = RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    
    models.append(model)
    cv_scores.append(mae)

    print(f"Fold {fold+1} MAE: {mae:.4f}")

print(f"\nAverage MAE across folds: {np.mean(cv_scores):.4f}")

Fold 1
Fold 1 MAE: 0.2025
Fold 2
Fold 2 MAE: 0.2019
Fold 3
Fold 3 MAE: 0.2001
Fold 4
Fold 4 MAE: 0.1998
Fold 5
Fold 5 MAE: 0.2023

Average MAE across folds: 0.2013


In [15]:
# -------------------- 9. 최종 모델 예측 (선택사항) --------------------
# 예를 들면, 가장 마지막 fold 모델을 사용하여 test셋 예측
final_preds = models[-1].predict(X_test_imputed)

In [16]:
final_preds

array([0.462, 0.482, 0.459, ..., 0.551, 0.482, 0.534])

In [19]:
len(test) == len(final_preds)

True

In [21]:
sample_submission['성공확률'] = final_preds
sample_submission.to_csv('./baseline_submission.csv', index = False)

In [22]:
sample_submission.head()

,ID,성공확률
0,TEST_0000,0.462
1,TEST_0001,0.482
2,TEST_0002,0.459
3,TEST_0003,0.485
4,TEST_0004,0.608
